[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ourownstory/neural_prophet/blob/master/example_notebooks/trend_peyton_manning.ipynb)

# Automatic trend changepoint selection
This is a continuation of the example notebook `trend_peyton_manning`, focusing on automatic changepoint selection.

We learned in that notebook how we can increase the flexibility of the trend, at the danger of overfitting.

Now, we will learn how to use regularization to avoid overfitting while maintaining high flexibility, resuting in something that could be described as 'automatic trend changepoint selection'.

In [1]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes
    data_location = "https://raw.githubusercontent.com/ourownstory/neural_prophet/master/"
else:
    data_location = "../"

In [2]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
# set_log_level("ERROR")

In [3]:
df = pd.read_csv(data_location + "example_data/wp_log_peyton_manning.csv")
# df.head(3)
lr = None
epochs = None

## Overfitted Trend
As this dataset has some strong seasonality, we would usually fit it with seasonality enabled.
For demonstration purposes, we manually disable all seasonalities here to show how Trend can overfit to seasonal patterns, and how to avoid this with regularization.

In [4]:
m = NeuralProphet(
    learning_rate=lr,
    epochs=epochs,
    n_changepoints=50,
    trend_reg=0,
    changepoints_range=0.90,    
    daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False,
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 91


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.37E-02, min: 2.02E+01
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 3.24E+00
Epoch[91/91]: 100%|██████████| 91/91 [00:09<00:00, 10.08it/s, SmoothL1Loss=0.00809, MAE=0.4, MSE=0.327, RegLoss=0]  


In [5]:
# fig1 = m.plot(forecast)
# fig2 = m.plot_parameters()

The regularized clearly overfits to the annual seasonal pattern - we can't really call this a 'trend'. Let's try addressing this by adding moderate regularization:

## Regularized Trend:
By adding regularization, we can achieve an automatic selection of the most relevant changepoints and draw the rate changes of other points close to zero. 


In [6]:
m = NeuralProphet(
    learning_rate=lr,
    epochs=epochs,
    n_changepoints=50,
    trend_reg=1,
    changepoints_range=0.90,    
    daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False,
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 91


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 4.99E-03, min: 1.76E+00
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 2.49E-01
Epoch[91/91]: 100%|██████████| 91/91 [00:08<00:00, 10.60it/s, SmoothL1Loss=0.0161, MAE=0.62, MSE=0.651, RegLoss=0.0831]  


In [7]:
# fig1 = m.plot(forecast)
# fig2 = m.plot_parameters()


Now the model selects fewer trend changepoints, drawing the rest closer to zero. But there is still room for improvement.

## Fine-tuning Trend Flexibility
We can adjust the regularization strength to get more or less points with a non-zero rate change.

Note: for too high regularization strengths, the model fitting process becomes unstable. If so, take manual control of learning rate and potentially more hyperparameters.

In [8]:
m = NeuralProphet(
    learning_rate=lr,
    epochs=epochs,
    n_changepoints=50,
    trend_reg=5,
    changepoints_range=0.90,   
    daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False,
)
metrics = m.fit(df, freq="D")
future = m.make_future_dataframe(df, n_historic_predictions=len(df))
forecast = m.predict(future)

INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 91


INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.89E-03, min: 1.90E+00
INFO - (NP.utils_torch.lr_range_test) - learning rate range test selected lr: 3.06E-01
Epoch[91/91]: 100%|██████████| 91/91 [00:08<00:00, 10.56it/s, SmoothL1Loss=0.0176, MAE=0.652, MSE=0.713, RegLoss=3.37e-5] 


In [9]:
# fig1 = m.plot(forecast)
# fig2 = m.plot_parameters()